In [6]:
import boto3
import time
ec2 = boto3.resource('ec2')
ec2_client = boto3.client('ec2')
autoscaling = boto3.client('autoscaling')
ecs = boto3.client('ecs')
launchtemplate = 'EC2ContainerService-sysvis-EcsInstanceLc-1H1654961ZY8F'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
vpc = 'vpc-0d7c412110e5f49f2'
subnet = 'subnet-0596881f608fd4f70'
securitygroup = 'sg-05c4123649a439051'
instance_id = ec2.create_instances(MinCount=1, MaxCount=1, LaunchTemplate={'LaunchTemplateName':launchtemplate}, SecurityGroupIds=[securitygroup], SubnetId=subnet)
instance_id = instance_id[0].instance_id
print(instance_id)
ready = False
time.sleep(2)
while not ready:
    if ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['State']['Name'] =='running':
        ready = True
time.sleep(2)
autoscaling.attach_instances(AutoScalingGroupName=autoscalinggroup, InstanceIds=[instance_id])
ready = False
print('created')
while not ready:
    response = autoscaling.describe_auto_scaling_groups(AutoScalingGroupNames=[autoscalinggroup])
    realresponse = response['AutoScalingGroups'][0]
    instances = realresponse['Instances']
    for i in instances:
        if i['InstanceId'] == instance_id:
            ready = True
print('attached')
ready = False
while not ready:
    containers = ecs.list_container_instances(cluster='sysvis')['containerInstanceArns']
    for i in containers:
        container = i
        findinstanceid = ecs.describe_container_instances(cluster='sysvis',containerInstances=[container])['containerInstances'][0]['ec2InstanceId']
        if findinstanceid == instance_id:
            ready = True
task = ecs.start_task(cluster='sysvis', taskDefinition='sysvis', containerInstances=[container])
time.sleep(2)
task_arn = task['tasks'][0]['taskArn']
ready = False
print('container')
while not ready:
    status = ecs.describe_tasks(cluster = 'sysvis', tasks = [task_arn])['tasks'][0]['lastStatus']
    if status == 'RUNNING':
        ready = True
print('finished')
print(ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicDnsName'])

i-075548e1c717b5347


created
attached


container


finished


ec2-54-226-46-180.compute-1.amazonaws.com


In [8]:
import boto3
autoscaling = boto3.client('autoscaling')
ec2_client = boto3.client('ec2')
instance_id = 'i-0f4a2d9e93d376362'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
autoscaling = boto3.client('autoscaling')
response = autoscaling.detach_instances(InstanceIds=[instance_id],AutoScalingGroupName=autoscalinggroup, ShouldDecrementDesiredCapacity=True)
print(response)
response = ec2_client.terminate_instances(InstanceIds=[instance_id]) 

{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-005c3cef18d2d2744', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '0e43735c-e2b2-4538-a70e-9423dff979fc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0e43735c-e2b2-4538-a70e-9423dff979fc', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'accept-encoding', 'date': 'Thu, 12 Nov 2020 13:46:48 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
